In [1]:
import tensorflow as tf
from tensorflow.keras import models, layers
import matplotlib.pyplot as plt

In [2]:
BATCH_SIZE = 32
IMAGE_SIZE = 256
CHANNELS=3
EPOCHS=50

In [5]:
dataset = tf.keras.preprocessing.image_dataset_from_directory(
    "C:/Users/User/Desktop/299/CSE-299-Project/Dataset/Disease dataset",
    seed=123,
    shuffle=True,
    image_size=(IMAGE_SIZE,IMAGE_SIZE),
    batch_size=BATCH_SIZE
)

Found 7025 files belonging to 7 classes.


In [6]:
class_names = dataset.class_names
class_names

['Corn Blight',
 'Corn Common_Rust',
 'Corn Gray_Leaf_Spot',
 'Corn Healthy',
 'Cotton Disease',
 'Wheat disease',
 'rice_leaf_diseases']